In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

- Based on the provided code, we want to forecast if a breast lump is malignant (cancer) or benign (non cancer).
- The target variable in the dataset is labelled as 'y' and holds the results of the breast mass categorization. It is a factor variable with two levels: "M" for malignant and "B" for benign.
- During the model training phase, we use characteristics collected from fine needle aspirate biopsy slides, such as the mean, standard error, and worst values of various nuclear parameters. These criteria include radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, and fractal dimension.

In [ ]:
data = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-diagnostic-dataset/brca.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
X = data.iloc[:, :-1]  
y = data.iloc[:, -1]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
feature_engineering = [('scaler', StandardScaler()),
                      ('feature_selection', SelectKBest(f_classif))]

In [ ]:
pipeline = Pipeline(feature_engineering + [('classifier', RandomForestClassifier())])

In [ ]:
param_grid = {'feature_selection__k': [5, 10, 15],
              'classifier__n_estimators': [50, 100, 200],
              'classifier__max_depth': [None, 5, 10]}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [ ]:
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))